In [3]:
!nvidia-smi

Sun May 22 05:00:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
!pip install -q transformers
!pip install -q datasets

     |████████████████████████████████| 4.2 MB 4.3 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 596 kB 24.0 MB/s 
     |████████████████████████████████| 6.6 MB 58.6 MB/s 
     |████████████████████████████████| 346 kB 4.3 MB/s 
     |████████████████████████████████| 212 kB 75.0 MB/s 
     |████████████████████████████████| 140 kB 70.2 MB/s 
     |████████████████████████████████| 1.1 MB 62.1 MB/s 
     |████████████████████████████████| 127 kB 75.8 MB/s 
     |████████████████████████████████| 271 kB 71.0 MB/s 
     |████████████████████████████████| 94 kB 4.4 MB/s 
     |████████████████████████████████| 144 kB 77.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
#transformers
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler 
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments

In [7]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

# Load Dataset


In [8]:
train_dataset_path = "/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/aug_clean_train.csv"
test_dataset_path = "/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_test.csv"

In [9]:
from datasets import Dataset, load_dataset

dataset = load_dataset('csv', data_files={'train': train_dataset_path,
                                          'val': test_dataset_path})

Using custom data configuration default-c23d9ec77aab9cc0


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-c23d9ec77aab9cc0/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

## Preprocess data

In [10]:
dataset = dataset.rename_column("text_clean", "texts")
dataset = dataset.rename_column("Sentiment", "labels")
# dataset['train'], dataset['validation'] = dataset['train'].train_test_split(0.2).values()

In [11]:
def label_mapping(examples):
    label_to_idx = {'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                    'Positive': 2, 'Extremely Positive': 2}
    examples["labels"] = label_to_idx[examples["labels"]]
    return examples
    
dataset = dataset.map(label_mapping)

  0%|          | 0/48483 [00:00<?, ?ex/s]

  0%|          | 0/3787 [00:00<?, ?ex/s]

In [12]:
def tokenizer_function(examples):
    result = tokenizer(examples["texts"], max_length=128,
                       padding='max_length', truncation=True)
    result["labels"] = examples["labels"]
    return result

tokenized_datasets = dataset.map(tokenizer_function, batched=True, remove_columns=['texts', 'labels'])

  0%|          | 0/49 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 48483
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3787
    })
})

# Roberta + LSTM

In [14]:
import torch 
from transformers import RobertaModel
from transformers.modeling_outputs import SequenceClassifierOutput

class CustomModel(torch.nn.Module):
    def __init__(self, checkpoint, num_labels):
        super(CustomModel, self).__init__()
        self.num_labels = num_labels
        self.criterion = torch.nn.CrossEntropyLoss()

        # Load model with given checkpoint and extract its body
        self.model = RobertaModel.from_pretrained(checkpoint, num_labels=self.num_labels)
        self.dropout = torch.nn.Dropout(0.1)
        self.lstm = torch.nn.LSTM(768, 256, batch_first=True, bidirectional=True)
        self.classifier = torch.nn.Linear(256*2, self.num_labels)
    
    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = self.dropout(outputs[0])

        lstm_output, (h, c) = self.lstm(sequence_output)
        hidden = torch.cat((lstm_output[:, -1, :256], lstm_output[:, 0, :256]), dim=-1)
        logits = self.classifier(hidden.view(-1, 256*2))

        loss = None
        if labels is not None:
            loss = self.criterion(logits.view(-1, self.num_labels), labels.view(-1))
        
        return SequenceClassifierOutput(loss=loss, logits=logits,
                                        hidden_states=outputs.hidden_states,
                                        attentions=outputs.attentions)

In [15]:
# model = CustomModel("/content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2", num_labels=3)

# Fine-tuning Roberta with Trainer API

In [16]:
from datasets import load_metric

def compute_metrics(eval_preds):
    accuracy_metric = load_metric("accuracy")
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    f1_metric = load_metric("f1")

    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")

    return {
        "accuracy": accuracy['accuracy'],
        "precision": precision['precision'],
        "recall": recall['recall'],
        "f1": f1['f1'],
    }

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "/content/output",
    num_train_epochs=10,
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model = 'f1',
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    save_strategy = "epoch",
    logging_steps=200
)

In [18]:
# trainer=Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'],
#     eval_dataset=tokenized_datasets['val'],
#     compute_metrics=compute_metrics,
# )

In [19]:
# trainer.train()

# Hyperparameter search

In [20]:
! pip install -q optuna
! pip install -q ray[tune]

     |████████████████████████████████| 308 kB 3.8 MB/s 
     |████████████████████████████████| 210 kB 92.7 MB/s 
     |████████████████████████████████| 81 kB 11.1 MB/s 
     |████████████████████████████████| 78 kB 8.6 MB/s 
     |████████████████████████████████| 49 kB 7.0 MB/s 
     |████████████████████████████████| 146 kB 90.6 MB/s 
     |████████████████████████████████| 112 kB 90.6 MB/s 
     |████████████████████████████████| 53.2 MB 98.2 MB/s 
     |████████████████████████████████| 4.1 MB 66.0 MB/s 
     |████████████████████████████████| 8.8 MB 68.6 MB/s 
     |████████████████████████████████| 125 kB 95.5 MB/s 
     |████████████████████████████████| 461 kB 50.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0+zzzcolab20220506162203 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [21]:
def model_init():
    return CustomModel("/content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2", num_labels=3)

In [22]:
from transformers import EarlyStoppingCallback

trainer=Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
)

loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /content/drive/M

In [ ]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 5e-6, 5e-5, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 5, 10),
    }

best_run = trainer.hyperparameter_search(n_trials=15, direction="maximize", hp_space=my_hp_space)

[I 2022-05-22 05:02:33,246] A new study created in memory with name: no-name-7270963a-6258-47b1-b59b-479a02e6525c
Trial:
loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_versio

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.642200,0.403634,0.857671,0.863355,0.829837,0.843280
2,0.320700,0.329338,0.895432,0.893975,0.874522,0.883280
3,0.227800,0.331436,0.894640,0.891007,0.875002,0.882307
4,0.169300,0.341672,0.897544,0.904487,0.872179,0.885854
5,0.126400,0.376485,0.897280,0.888229,0.879980,0.883916
6,0.095400,0.395021,0.886454,0.862385,0.872019,0.866851
7,0.070000,0.480591,0.885926,0.871558,0.867156,0.868775


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128


Saving model checkpoint to /content/output/run-0/checkpoint-379
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-0/checkpoint-758
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-0/checkpoint-1137
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-0/checkpoint-1516
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-0/checkpoint-1895
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num ex

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.761200,0.478727,0.813309,0.802071,0.790150,0.794885
2,0.411700,0.385315,0.866385,0.864585,0.840039,0.850689
3,0.315200,0.372695,0.870610,0.870878,0.844520,0.855772
4,0.258900,0.346979,0.877476,0.870757,0.858680,0.863988
5,0.213700,0.348744,0.883285,0.882254,0.855119,0.866627
6,0.185000,0.358173,0.883285,0.877719,0.859257,0.867270
7,0.161900,0.378874,0.884869,0.876787,0.859527,0.867314
8,0.144700,0.405101,0.878004,0.868447,0.849753,0.858070
9,0.135200,0.409288,0.881701,0.869846,0.857015,0.862939


...

***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-6/checkpoint-379
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-6/checkpoint-758
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-6/checkpoint-1137
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-6/checkpoint-1516
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-6/checkpoint-1895
Trainer.model is not a `PreTrainedMod

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.867600,0.593333,0.762609,0.762063,0.722201,0.736259


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
[I 2022-05-22 12:15:25,311] Trial 7 pruned. 
Trial:
loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_ver

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.708500,0.434394,0.842355,0.838046,0.818150,0.826652


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 128
Saving model checkpoint to /content/output/run-8/checkpoint-379
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
